In [1]:
import pandas as pd
import numpy as np

print("🚀 Starting Data Integration (Final Version)...")

# ==========================================
# 1. โหลดข้อมูลงานวิจัยจุฬาฯ (Main Data)
# ==========================================
try:
    df_papers = pd.read_csv('papers_all_years.csv')
    
    # เลือกคอลัมน์ที่ต้องใช้ (รวม subject_areas_str ด้วย เพื่อไม่ให้ Error ทีหลัง)
    required_cols = ['title', 'journal', 'year', 'citedby_count', 'countries_str', 'subject_areas_str']
    
    # ตรวจสอบว่ามีคอลัมน์ครบไหม
    available_cols = [c for c in required_cols if c in df_papers.columns]
    df_papers = df_papers[available_cols].dropna(subset=['journal'])
    
    print(f"   -> Chula Papers loaded: {len(df_papers)} rows")
    
except FileNotFoundError:
    print("❌ Error: ไม่พบไฟล์ papers_all_years.csv")
    exit()

# ==========================================
# 2. โหลดข้อมูล SJR Global (External Data)
# ==========================================
try:
    # อ่านไฟล์ Scimago (SJR)
    df_sjr = pd.read_csv('scimagojr 2023.csv', sep=';', quotechar='"', on_bad_lines='skip')
    
    # เลือกเฉพาะชื่อและ Quartile
    df_sjr = df_sjr[['Title', 'SJR Best Quartile']]
    
    # Clean ชื่อวารสาร (ลบเครื่องหมายคำพูดและช่องว่าง)
    df_sjr['Title'] = df_sjr['Title'].str.replace('"', '', regex=False).str.strip()
    
    print(f"   -> SJR Global Data loaded: {len(df_sjr)} journals")
    
except FileNotFoundError:
    print("❌ Error: ไม่พบไฟล์ scimagojr 2023.csv")
    exit()

# ==========================================
# 3. การรวมข้อมูล (Merging)
# ==========================================
print("🔄 Merging datasets...")

# สร้างคอลัมน์ชั่วคราวเป็นตัวพิมพ์เล็กเพื่อเทียบชื่อ (Case-insensitive match)
df_papers['journal_lower'] = df_papers['journal'].str.lower().str.strip()
df_sjr['title_lower'] = df_sjr['Title'].str.lower().str.strip()

# เชื่อมข้อมูล
merged_df = df_papers.merge(df_sjr, left_on='journal_lower', right_on='title_lower', how='left')

# ==========================================
# 4. แสดงผลลัพธ์ (Statistics Report)
# ==========================================
total_papers = len(df_papers)
found_match = merged_df['SJR Best Quartile'].notna().sum()
q1_papers = merged_df[merged_df['SJR Best Quartile'] == 'Q1'].shape[0]

print("\n" + "="*40)
print("📊 INTEGRATION REPORT")
print("="*40)
print(f"Total Papers (Chula):    {total_papers}")
print(f"Matched with SJR:        {found_match}")
print(f"Match Success Rate:      {found_match/total_papers*100:.2f}%")
print("-" * 40)
print(f"🏆 Found Q1 Papers:      {q1_papers}")
print(f"🥈 Found Q2 Papers:      {merged_df[merged_df['SJR Best Quartile'] == 'Q2'].shape[0]}")
print(f"🥉 Found Q3 Papers:      {merged_df[merged_df['SJR Best Quartile'] == 'Q3'].shape[0]}")
print(f"🔻 Found Q4 Papers:      {merged_df[merged_df['SJR Best Quartile'] == 'Q4'].shape[0]}")
print("="*40 + "\n")

# ==========================================
# 5. บันทึกไฟล์ (สำคัญ: ต้องสร้าง is_Q1)
# ==========================================
# ลบคอลัมน์ช่วยทิ้ง
merged_df.drop(columns=['journal_lower', 'title_lower', 'Title'], inplace=True, errors='ignore')

# *** เพิ่มบรรทัดนี้ครับ ***
# สร้างคอลัมน์ Target สำหรับ AI (1=Q1, 0=Others)
merged_df['is_Q1'] = merged_df['SJR Best Quartile'].apply(lambda x: 1 if x == 'Q1' else 0)

# สร้างไฟล์ผลลัพธ์
output_filename = 'chula_papers_with_quality.csv'
merged_df.to_csv(output_filename, index=False)
print(f"💾 Saved integrated data to '{output_filename}' (with 'is_Q1' column)")

🚀 Starting Data Integration (Final Version)...
   -> Chula Papers loaded: 20216 rows
   -> SJR Global Data loaded: 31283 journals
🔄 Merging datasets...

📊 INTEGRATION REPORT
Total Papers (Chula):    20216
Matched with SJR:        18537
Match Success Rate:      91.69%
----------------------------------------
🏆 Found Q1 Papers:      9680
🥈 Found Q2 Papers:      4577
🥉 Found Q3 Papers:      2461
🔻 Found Q4 Papers:      1260

💾 Saved integrated data to 'chula_papers_with_quality.csv' (with 'is_Q1' column)
